In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

import librosa
import librosa.display

from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from my_functions.my_fun import *
from my_functions.my_models import *
from my_functions.My_nn import My_nn

In [2]:
load_dotenv()
ASSETS_DIR = os.getenv('ASSETS_DIR')

# Preparar las etiquetas

Cargar archivo de anotaciones y comprobar que existe

In [ ]:
anot = os.getenv('PMEMO_ANOTATIONS')
stat_anot_file = os.path.join(anot,'static_annotations.csv')

if os.path.exists(stat_anot_file):
    print("El archivo existe")
else:
    print("No hay no existe")
    
anotaciones_df = pd.read_csv(stat_anot_file)
anotaciones_df.head(5)

In [ ]:
labels = anotaciones_df[['Arousal(mean)','Valence(mean)']].values
print(labels)

# Preparar los espectrogramas

Cargar los espectrogramas ajustados

In [5]:
spectrograms = load_spectrograms(filename='spectrograms_down_padd')

In [ ]:
spectrograms = np.array(spectrograms)
print(type(spectrograms))
print(type(spectrograms[0]))
print(spectrograms[0].shape)

In [ ]:
spectrograms = np.expand_dims(spectrograms, axis=-1)
print(spectrograms.shape)
print(spectrograms[0].shape)

# Dividir los datos

In [8]:
#Entrenamiento y pruebas
X_train,X_test,y_train,y_test = train_test_split( spectrograms,
                                                 labels,
                                                 test_size=0.2,
                                                 random_state=83)
#Entrenamiento y validación
X_train,X_val, y_train,y_val =  train_test_split( X_train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=83)

# Normalizar?

In [ ]:
# Asumiendo que `spectrograms_array` es tu array de espectrogramas
print("Valor mínimo:", np.min(spectrograms))
print("Valor máximo:", np.max(spectrograms))
print("Media:", np.mean(spectrograms))
print("Desviación estándar:", np.std(spectrograms))

In [ ]:
# Aplana el array para facilitar la visualización
values = spectrograms.flatten()

plt.hist(values, bins=50)
plt.title('Histograma de Valores de Espectrogramas')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Obtener un resumen estadístico
stats = {
    'Mínimo': np.min(values),
    'Máximo': np.max(values),
    'Media': np.mean(values),
    'Desviación Estándar': np.std(values),
    '25º Percentil': np.percentile(values, 25),
    '50º Percentil (Mediana)': np.percentile(values, 50),
    '75º Percentil': np.percentile(values, 75)
}

for key, value in stats.items():
    print(f'{key}: {value}')

In [12]:
# Normalizar los datos
X_train_norm = X_train / np.max(X_train)
X_val_norm = X_val / np.max(X_val)
X_test_norm = X_test / np.max(X_test)

In [ ]:
# Graficar el espectrograma de la primer canción
aux_spec = np.squeeze(X_train[1])


#Visualizar
fig,ax = plt.subplots(figsize=(10,5))

img = librosa.display.specshow(aux_spec,
                               x_axis='time',
                               y_axis='log',
                               ax=ax)

ax.set_title('Espectrograma de la primer canción',fontsize=(20))
fig.colorbar(img,ax=ax,format=f'%0.2f')

plt.show()

In [ ]:
# Graficar el espectrograma de la primer canción pero normalizada
espectrograma_normalizado = np.squeeze(X_train_norm[1])
# Visualizar el espectrograma normalizado
fig,ax = plt.subplots(figsize=(10,5))

img = librosa.display.specshow(espectrograma_normalizado,
                               x_axis='time',
                               y_axis='log',
                               ax=ax)
ax.set_title('Espectrograma de la primer canción normalizada',fontsize=(20))
fig.colorbar(img, ax=ax, format=f'%0.2f')
plt.show()

In [15]:
# Normalizar los datos
X_train = X_train_norm
X_val = X_val_norm
X_test = X_test_norm

# Guardar datos procesados

In [16]:
import h5py

In [17]:
# Guardar los datos en un archivo .h5
DATOS_DIR = os.path.join(ASSETS_DIR,'processed_data.h5')

with h5py.File(DATOS_DIR, 'w') as hf:
    hf.create_dataset('X_train', data=X_train)
    hf.create_dataset('X_test', data=X_test)
    hf.create_dataset('X_val', data=X_val)
    hf.create_dataset('y_train', data=y_train)
    hf.create_dataset('y_test', data=y_test)
    hf.create_dataset('y_val', data=y_val)

# Cargar datos procesados

In [3]:
import h5py
# Cargar los datos desde un archivo .h5
DATOS_DIR = os.path.join(ASSETS_DIR,'processed_data.h5')
with h5py.File(DATOS_DIR, 'r') as hf:
    X_train = hf['X_train'][:]
    X_test = hf['X_test'][:]
    X_val = hf['X_val'][:]
    y_train = hf['y_train'][:]
    y_test = hf['y_test'][:]
    y_val = hf['y_val'][:]

# CNN

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [5]:
print(f'--------------------ENTRENAMIENTO--------------------------------')
print(f'Dimensión datos de entrenamiento: {X_train.shape}')
print(f'Dimensión etiquetas de entrenamiento: {y_train.shape}')
print(f'--------------------PRUEBAS--------------------------------')
print(f'Dimensión datos de prueba: {X_test.shape}')
print(f'Dimensión etiquetas de prueba: {y_test.shape}')
print(f'--------------------VALIDACIÓN--------------------------------')
print(f'Dimensión datos de validación: {X_val.shape}')
print(f'Dimensión etiquetas de validación: {y_val.shape}')

--------------------ENTRENAMIENTO--------------------------------
Dimensión datos de entrenamiento: (490, 513, 1671, 1)
Dimensión etiquetas de entrenamiento: (490, 2)
--------------------PRUEBAS--------------------------------
Dimensión datos de prueba: (154, 513, 1671, 1)
Dimensión etiquetas de prueba: (154, 2)
--------------------VALIDACIÓN--------------------------------
Dimensión datos de validación: (123, 513, 1671, 1)
Dimensión etiquetas de validación: (123, 2)


In [6]:
modelo1 = modelo1()
primer_exp = My_nn(X_train=X_train,y_train=y_train,
                   X_test=X_test,y_test=y_test,
                   X_val=X_val,y_val=y_val,
                   model=modelo1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 513, 1671, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, 513, 1671, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 835, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 256, 835, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 835, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 417, 64)     0     

c:\Users\leo_d\anaconda3\envs\mirtools\lib\site-packages\visualkeras\layered.py:86: UserWarning: The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.
  warnings.warn("The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.")


entrenar el modelo

In [7]:
hist =  primer_exp.train()

TypeError: fit() got an unexpected keyword argument 'batchsize'